## Cifar10 Classification 정규화 적용

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

#### GPU 메모리 설정 (프로그램에서 필요한만큼만 할당하는 방식)

In [ ]:
# 런타임에서 할당하는데 필요한 양만큼의 GPU 메모리를 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

### 패션 MNIST 데이터셋 임포트하기

load_data() 함수를 호출해서 Fashion MNIST 로딩 : 네 개의 넘파이(NumPy) 배열이 반환

In [ ]:
cifar10 = keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
print(train_labels.shape)

train_labels = train_labels.flatten()
test_labels = test_labels.flatten()

In [ ]:
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

### 데이터 탐색

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
train_labels

In [ ]:
len(test_labels)

### 데이터 전처리

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

픽셀 값을 [0,255]에서 [0,1]로 조정

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Traning Set 25개 이미지, 클래스 이름 출력

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i]])
plt.show()

### 모델 구성
* Activation Function 변경 (relu에서 selu로)
* Weight Decay (L2 regularizer, weight_decay = 1e-4)
* Dropout 적용 (MaxPooling 다음에 추가 (drop rate는 0.2, 0.3, 0.4 순으로 적용))

In [ ]:
from tensorflow.keras import regularizers
weight_decay = 1e-4

model = keras.Sequential([
      keras.layers.Conv2D(32, 3, padding="SAME", activation='selu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(32,32,3)),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(32, 3, padding="SAME", activation='selu', kernel_regularizer=regularizers.l2(weight_decay)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPooling2D(),
      keras.layers.Dropout(0.2),
      keras.layers.Conv2D(64, 3, padding="SAME", activation='selu', kernel_regularizer=regularizers.l2(weight_decay)),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(64, 3, padding="SAME", activation='selu', kernel_regularizer=regularizers.l2(weight_decay)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPooling2D(),
      keras.layers.Dropout(0.3),
      keras.layers.Conv2D(64, 3, padding="SAME", activation='selu', kernel_regularizer=regularizers.l2(weight_decay)),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(64, 3, padding="SAME", activation='selu', kernel_regularizer=regularizers.l2(weight_decay)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPooling2D(),
      keras.layers.Dropout(0.4),
      keras.layers.Flatten(),
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dense(10, activation='softmax')])

In [1]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

NameError: name 'model' is not defined

### 모델 훈련
* epoch 수 200으로 증가
* Data Augmentation (Rotation 15°, Width Shift 10%, Height Shift 10%, Horizontal Flip)
* Learning Rate Scheduling

In [ ]:
EPOCHS = 200
batch_size = 64

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# Learning Rate Schedule
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
lr_schedule_cb = keras.callbacks.LearningRateScheduler(lr_schedule)

# ImageDataGenerator  shear_range=0.2, zoom_range=0.2,
train_generator= tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

train_generator.fit(train_images)

# Model Fit
model.fit(train_generator.flow(train_images,
                               train_labels,
                               batch_size=batch_size),
          epochs=EPOCHS,
          validation_data=(test_images,test_labels),
          callbacks=[lr_schedule_cb])

### 테스트 성능 측정 (정확도)

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\n테스트 정확도:', test_acc)

### 예측 만들기

In [ ]:
predictions = model.predict(test_images)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# 처음 X 개의 테스트 이미지와 예측 레이블, 진짜 레이블을 출력합니다
# 올바른 예측은 파랑색으로 잘못된 예측은 빨강색으로 나타냅니다
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()